In [1]:
import os
import sys
os.chdir('..')
sys.path.append(os.getcwd())

In [2]:
import torch
from Models.pretrain import *
from Models.cPB import cPB
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
import argparse
from torch.autograd import Variable
import pprint
import copy
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=UserWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=DeprecationWarning, module='torch.storage')

In [4]:
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return np.trunc(stepper * number) / stepper

def print_stats(data_list, label):
    mean_val = np.mean(data_list)
    std_val = np.std(data_list)

    # Truncate values instead of rounding
    truncated_mean = truncate(mean_val, 3)
    truncated_std = truncate(std_val, 3)
    print(f"{label} Mean: {truncated_mean}%")
    print(f"{label} Std:  {truncated_std}%")
    print(f"{label} complete vale:  {truncated_mean}+{truncated_std}")
    print(10 * '*')
    
def replication(data1):
  kappa = data1

  # Apply the filtering
  before_50 = kappa[:50]  # Elements up to index 50 (exclusive)
  after_50 = kappa[50:]
  after_50_filtered = after_50[::2]
  filtered_kappa = np.concatenate((before_50, after_50_filtered))
  return filtered_kappa

# Weather datasets


## CONFIGURATION:


In [19]:
base_model = 'GRU'
batch_size = 128
hidden_size = 50
seq_len = 10
epoch_size=10
lr = 0.01
number_of_tasks=4
mask_selection_NofBatch = 50
input_size=2
iteration=1
mask_init='uniform'
dataset='SINE'
dataset_name = "sine_rw10_mode5_extended_16-16_1234"
Pretrain_task = '1'
df = pd.read_csv(os.path.join(f"datasets/{dataset}/", f"{dataset_name}.csv"))
pretrain_model_addr=f'Performance/Pretrain/{base_model}/{dataset}/After/{dataset}-Task_{Pretrain_task}-{base_model}-pretrain-hidden{hidden_size}-epoch10_iter{iteration}.pickle'
mask_weights=[] #if we have initial masks then reload it here


mask_option = 'SUM'
model_type = 'cPB'
batch_first = True
low_rank = False

In [10]:
base_model = 'GRU'
batch_size = 128
hidden_size = 50
seq_len = 11
epoch_size=10
lr = 0.01
number_of_tasks=4
mask_selection_NofBatch = 50
input_size=4
mask_init='uniform'
dataset='Weather'
dataset_name = "weather_st124_1conf"
Pretrain_task = '1'
mask_option = 'SUM'
model_type = 'cPB'

In [11]:
batch_50_1_1 = []
batch_50_2_1 = []
batch_50_3_1 = []
batch_50_4_1 = []
end_1_1 = []
end_2_1 = []
end_3_1 = []
end_4_1 = []

all_results_2 = [[] for i in range(0,8)]
for i in range (1,11):
  file_1 = f'Performance/Results/{model_type}/{dataset}/Periodic/pretrain_T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}-iter{i}.pkl'
  with open(file_1, 'rb') as f:
    data1 = pickle.load(f)

  all_results_2[0].append(np.mean(data1['task_1']['kappa'][0:50]))
  out = replication(data1['task_1']['kappa'][:])
  all_results_2[1].append(np.mean(out))
  all_results_2[2].append(np.mean(data1['task_2']['kappa'][0:50]))
  out = replication(data1['task_2']['kappa'][:])
  all_results_2[3].append(np.mean(out))
  all_results_2[4].append(np.mean(data1['task_3']['kappa'][0:50]))
  out = replication(data1['task_3']['kappa'][:])
  all_results_2[5].append(np.mean(out))
  all_results_2[6].append(np.mean(data1['task_4']['kappa'][0:50]))
  out = replication(data1['task_4']['kappa'][:])
  all_results_2[7].append(np.mean(out))


print_stats(all_results_2[0], "Batch 50-1")
print_stats(all_results_2[1], "End 1")
print_stats(all_results_2[2], "Batch 50-2")
print_stats(all_results_2[3], "End 2")
print_stats(all_results_2[4], "Batch 50-3")
print_stats(all_results_2[5], "End 3")
print_stats(all_results_2[6], "Batch 50-4")
print_stats(all_results_2[7], "End 4")


Batch 50-1 Mean: 0.636%
Batch 50-1 Std:  0.01%
Batch 50-1 complete vale:  0.636+0.01
**********
End 1 Mean: 0.663%
End 1 Std:  0.007%
End 1 complete vale:  0.663+0.007
**********
Batch 50-2 Mean: 0.702%
Batch 50-2 Std:  0.01%
Batch 50-2 complete vale:  0.702+0.01
**********
End 2 Mean: 0.735%
End 2 Std:  0.006%
End 2 complete vale:  0.735+0.006
**********
Batch 50-3 Mean: 0.392%
Batch 50-3 Std:  0.019%
Batch 50-3 complete vale:  0.392+0.019
**********
End 3 Mean: 0.567%
End 3 Std:  0.026%
End 3 complete vale:  0.567+0.026
**********
Batch 50-4 Mean: 0.734%
Batch 50-4 Std:  0.021%
Batch 50-4 complete vale:  0.734+0.021
**********
End 4 Mean: 0.751%
End 4 Std:  0.014%
End 4 complete vale:  0.751+0.014
**********
